In [12]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
import collections
from sklearn.model_selection import train_test_split
from nltk.metrics import precision, recall, f_measure

# Load the dataset
df = pd.read_csv('train.csv')

# Create a mapping from metaphorID to the actual metaphor words
metaphor_mapping = {
    0: 'road',
    1: 'candle',
    2: 'light',
    3: 'spice',
    4: 'ride',
    5: 'train',
    6: 'boat'
}

# Apply the mapping
df['metaphor_word'] = df['metaphorID'].map(metaphor_mapping)

# Convert the label to integers
df['label_boolean'] = df['label_boolean'].astype(int)

# Preprocess text and create features
def preprocess_text(text):
    tokens = word_tokenize(text)
    return [word.lower() for word in tokens if word.isalpha() and word not in stopwords.words('english')]

def create_features(text, metaphor_word):
    tokens = preprocess_text(text)
    features = {token: True for token in tokens}
    features['METAPHOR_WORD_' + metaphor_word] = True
    return features

data = [(create_features(row['text'], row['metaphor_word']), row['label_boolean']) for index, row in df.iterrows()]

# Split the dataset
train_set, val_set = train_test_split(data, test_size=0.2, random_state=42)

# Train the classifier
classifier = NaiveBayesClassifier.train(train_set)

# Function to calculate classification metrics
def classification_report(classifier, dataset, dataset_name):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(dataset):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)

    print(f'Model: NLTK Naive Bayes Classifier - {dataset_name}')
    for label in refsets:
        print(f'Label: {label}')
        p = precision(refsets[label], testsets[label])
        r = recall(refsets[label], testsets[label])
        f = f_measure(refsets[label], testsets[label])
        print(f'Precision: {p:.2f}, Recall: {r:.2f}, F1-Score: {f:.2f}')

    accuracy = nltk_accuracy(classifier, dataset)
    print(f'Accuracy: {accuracy:.4f}\n')

# Classification report for the training set
classification_report(classifier, train_set, "Training Set")

# Classification report for the testing set
classification_report(classifier, val_set, "Testing Set")

Model: NLTK Naive Bayes Classifier - Training Set
Label: 1
Precision: 1.00, Recall: 0.45, F1-Score: 0.62
Label: 0
Precision: 0.35, Recall: 0.99, F1-Score: 0.52
Accuracy: 0.5769

Model: NLTK Naive Bayes Classifier - Testing Set
Label: 1
Precision: 0.95, Recall: 0.31, F1-Score: 0.47
Label: 0
Precision: 0.33, Recall: 0.95, F1-Score: 0.49
Accuracy: 0.4813

